In [1]:
import io
from io import StringIO
import os
import sys
import cv2
from pyspark import SparkContext
from pyspark.ml.feature import CountVectorizer
from pyspark.sql import Row, SparkSession 
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, FloatType, StringType, IntegerType
import pyspark.sql.functions as F
# for visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
data_path = "./coronahack-chest-xraydataset/Chest_xray_Corona_Metadata.csv"
meta_data = spark.read.format('csv').options(header='true', inferschema='true').load(data_path)

In [4]:
meta_data.show(5)

+---+-----------------+------+------------+----------------------+----------------------+
|_c0| X_ray_image_name| Label|Dataset_type|Label_2_Virus_category|Label_1_Virus_category|
+---+-----------------+------+------------+----------------------+----------------------+
|  0|IM-0128-0001.jpeg|Normal|       TRAIN|                  null|                  null|
|  1|IM-0127-0001.jpeg|Normal|       TRAIN|                  null|                  null|
|  2|IM-0125-0001.jpeg|Normal|       TRAIN|                  null|                  null|
|  3|IM-0122-0001.jpeg|Normal|       TRAIN|                  null|                  null|
|  4|IM-0119-0001.jpeg|Normal|       TRAIN|                  null|                  null|
+---+-----------------+------+------------+----------------------+----------------------+
only showing top 5 rows



In [5]:
meta_data_with_labels = meta_data.withColumn(
    'new_label',
    F.when((F.col("Label")  =="Normal"), 0).otherwise(1)
)

In [7]:
number_of_samples = meta_data_with_labels.count()
train_set = meta_data_with_labels.where(meta_data.Dataset_type == "TRAIN")
test_set = meta_data_with_labels.where(meta_data.Dataset_type == "TEST")
num_train = train_set.count()
num_test = test_set.count()

print(f"Number of samples: ",number_of_samples )
print(f"Train samples: ", num_train)
print(f"Test samples: ",num_test)

Number of samples:  5910
Train samples:  5286
Test samples:  624


In [8]:
train_path = os.getcwd() +'/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train/'
@udf(returnType=StringType())
def get_absolute_path(img_file):
    abs_path = train_path+str(img_file)
    return abs_path
train = train_set.withColumn("image_path", get_absolute_path(col("X_ray_image_name")))

test_path = os.getcwd() +'/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test/'
@udf(returnType=StringType())
def get_test_absolute_path(img_file):
    abs_path = test_path+str(img_file)
    return abs_path
test = test_set.withColumn("image_path", get_test_absolute_path(col("X_ray_image_name")))



In [9]:
# TODO Only selecting the 3 colums,we need. Can change later on
select_train = train.select("image_path", "new_label")
select_test = test.select("image_path", "new_label")
# select_train.show(4)
# select_test.show(4)
print(select_train.count())
print(select_test.count())


5286
624


# Convert Images to Orb Descriptors

In [10]:
udf_image = udf(lambda img: get_desc(img), ArrayType(FloatType()))

def fetch_descriptor(img):
    
    image = cv2.imread(img)
    image = cv2.resize(image, (700, 700))
    orb_cv = cv2.ORB_create(nfeatures=50)
    keypts_for_orb, descriptor = orb_cv.detectAndCompute(image, None)
    

    if descriptor is None:
        descriptor = []
    else:
        descriptor = descriptor.flatten().tolist()
    return descriptor

udf_image = udf(fetch_descriptor, ArrayType(IntegerType()))

# TODO need to do for test data as well
# change afterwards
train_descriptor = select_train.withColumn("descriptors", udf_image("image_path"))
train_descriptor = train_descriptor.filter(train_descriptor.descriptors. isNotNull())


test_descriptor = select_test.withColumn("descriptors", udf_image("image_path"))
test_descriptor = test_descriptor.filter(test_descriptor.descriptors. isNotNull())



train_descriptor.show(4)
test_descriptor.show(4)
print(train_descriptor.count())
print(test_descriptor.count())



+--------------------+---------+--------------------+
|          image_path|new_label|         descriptors|
+--------------------+---------+--------------------+
|/Users/mma525/Doc...|        0|[73, 50, 24, 32, ...|
|/Users/mma525/Doc...|        0|[179, 225, 108, 6...|
|/Users/mma525/Doc...|        0|[252, 23, 246, 95...|
|/Users/mma525/Doc...|        0|[197, 237, 172, 1...|
+--------------------+---------+--------------------+
only showing top 4 rows

+--------------------+---------+--------------------+
|          image_path|new_label|         descriptors|
+--------------------+---------+--------------------+
|/Users/mma525/Doc...|        0|[165, 248, 133, 8...|
|/Users/mma525/Doc...|        0|[140, 159, 122, 2...|
|/Users/mma525/Doc...|        0|[2, 174, 170, 185...|
|/Users/mma525/Doc...|        0|[188, 255, 94, 78...|
+--------------------+---------+--------------------+
only showing top 4 rows

5286
624


In [11]:
from pyspark.sql.functions import size

train_filtered = train_descriptor.filter(size('descriptors')==1600)
test_filtered = test_descriptor.filter(size('descriptors')==1600)


print("Total train examples: ",train_filtered.count())
print("Total test examples: ",test_filtered.count())

Total train examples:  3769
Total test examples:  465


In [12]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())

train_with_vec = train_filtered.withColumn("vec_descriptors", list_to_vector_udf("descriptors"))
test_with_vec = test_filtered.withColumn("vec_descriptors", list_to_vector_udf("descriptors"))

print("Total train examples: ",train_with_vec.count())
print("Total test examples: ",test_with_vec.count())


Total train examples:  3769
Total test examples:  465


Split the training into 90/10 into train and validation data

In [13]:
train, val = train_with_vec.randomSplit([0.9, 0.1])
train.show()
val.show()
print("Total train split: ",train.count())
print("Total validation split: ",val.count())

+--------------------+---------+--------------------+--------------------+
|          image_path|new_label|         descriptors|     vec_descriptors|
+--------------------+---------+--------------------+--------------------+
|/Users/mma525/Doc...|        1|[65, 249, 57, 34,...|[65.0,249.0,57.0,...|
|/Users/mma525/Doc...|        1|[77, 108, 175, 95...|[77.0,108.0,175.0...|
|/Users/mma525/Doc...|        1|[124, 78, 132, 19...|[124.0,78.0,132.0...|
|/Users/mma525/Doc...|        1|[161, 4, 79, 85, ...|[161.0,4.0,79.0,8...|
|/Users/mma525/Doc...|        1|[213, 76, 159, 22...|[213.0,76.0,159.0...|
|/Users/mma525/Doc...|        1|[35, 172, 97, 13,...|[35.0,172.0,97.0,...|
|/Users/mma525/Doc...|        1|[195, 222, 238, 3...|[195.0,222.0,238....|
|/Users/mma525/Doc...|        1|[191, 59, 107, 64...|[191.0,59.0,107.0...|
|/Users/mma525/Doc...|        1|[124, 20, 243, 76...|[124.0,20.0,243.0...|
|/Users/mma525/Doc...|        1|[224, 161, 185, 1...|[224.0,161.0,185....|
|/Users/mma525/Doc...|   

# Logistic Regression Classifier

### Training

Define our model and train on the training data

In [14]:
from pyspark.ml.classification import LogisticRegression
logistic_regression = LogisticRegression(featuresCol = 'vec_descriptors', labelCol = 'new_label', maxIter=100)
logistic_regression_model = logistic_regression.fit(train)

### Evaluating on training data

In [15]:
predictions_train = logistic_regression_model.transform(train)
predictions_train.select("image_path","new_label","probability", "prediction").show(4)

+--------------------+---------+--------------------+----------+
|          image_path|new_label|         probability|prediction|
+--------------------+---------+--------------------+----------+
|/Users/mma525/Doc...|        1|[0.13442505324830...|       1.0|
|/Users/mma525/Doc...|        1|[1.92258211262361...|       1.0|
|/Users/mma525/Doc...|        1|[9.82620723655403...|       1.0|
|/Users/mma525/Doc...|        1|[3.55770393553846...|       1.0|
+--------------------+---------+--------------------+----------+
only showing top 4 rows



##### ROC evaluator

In [16]:
# Here we evaluate
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='new_label', metricName='areaUnderROC')
print('Train Area Under ROC: ', evaluator.evaluate(predictions_train))


Train Area Under ROC:  0.9999924207973321


##### Classifier evaluator

In [17]:

# multiclassclassification evaluator is used in order to measure accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='new_label', metricName='accuracy')
print('Train Accuracy: ', evaluator.evaluate(predictions_train))




Train Accuracy:  0.999117387466902


### Evaluating on validation data

In [18]:
predictions_val = logistic_regression_model.transform(val)
predictions_val.select("image_path","new_label","probability", "prediction").show(4)

+--------------------+---------+--------------------+----------+
|          image_path|new_label|         probability|prediction|
+--------------------+---------+--------------------+----------+
|/Users/mma525/Doc...|        1|[4.78632757097569...|       1.0|
|/Users/mma525/Doc...|        1|[1.57464035524034...|       1.0|
|/Users/mma525/Doc...|        0|[0.96248971097563...|       0.0|
|/Users/mma525/Doc...|        0|[3.80229589954006...|       1.0|
+--------------------+---------+--------------------+----------+
only showing top 4 rows



##### ROC evaluator

In [19]:
# Here we evaluate
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='new_label', metricName='areaUnderROC')
print('Val Area Under ROC: ', evaluator.evaluate(predictions_val))


Val Area Under ROC:  0.7197338830584702


##### Classifier evaluator

In [20]:

# multiclassclassification evaluator is used in order to measure accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='new_label', metricName='accuracy')
print('Val Accuracy: ', evaluator.evaluate(predictions_val))




Val Accuracy:  0.6513513513513514


### Evaluating on test data

We evaluate on the test data

In [21]:
test = test_with_vec
predictions_test = logistic_regression_model.transform(test)
predictions_test.select("image_path","new_label","probability", "prediction").show(4)

+--------------------+---------+--------------------+----------+
|          image_path|new_label|         probability|prediction|
+--------------------+---------+--------------------+----------+
|/Users/mma525/Doc...|        0|[3.78053985190336...|       1.0|
|/Users/mma525/Doc...|        0|[3.21354721515507...|       1.0|
|/Users/mma525/Doc...|        0|[0.99999999999451...|       0.0|
|/Users/mma525/Doc...|        0|[5.11732252417787...|       1.0|
+--------------------+---------+--------------------+----------+
only showing top 4 rows



##### ROC evaluator

In [22]:
# Here we evaluate
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='new_label', metricName='areaUnderROC')
print('Test Area Under ROC: ', evaluator.evaluate(predictions_test))

Test Area Under ROC:  0.6741709230883115


##### Classifier evaluator

In [23]:
# multiclassclassification evaluator is used in order to measure accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='new_label', metricName='accuracy')
print('Test Accuracy: ', evaluator.evaluate(predictions_test))

Test Accuracy:  0.6387096774193548
